# Assignment 2

In this assignment, we're set to embark on an exciting journey to the cutting edge of machine learning methodologies. Specifically, we'll explore and apply state-of-the-art fine-tuning techniques to large language models. The techniques we'll delve into are not only robust but also resource-efficient, allowing us to perform the task of fine-tuning even on the free T4 GPUs available in a Kaggle notebook.

Among the techniques we will explore is **Low-Rank Adaptation (LoRA)**, a novel method that has proven to be efficient and effective in adapting large pre-trained language models to specific tasks. LoRA is grounded in the hypothesis that updates to the weights during adaptation have a low "intrinsic rank", allowing us to constrain weight updates and reduce computational complexity, while preserving model performance.

Complementing LoRA, we will also engage with **mixed-precision training**. This technique combines different numerical precisions to perform computations, aiming to maximize the computational power of modern GPUs. Mixed-precision training can accelerate model training, reduce memory requirements, and thus enable us to train larger, more powerful models.

Finally, we will delve into **distributed training**, a must-know technique for handling very large models or datasets. With distributed training, we can leverage multiple GPUs or even multiple machines to collectively train a single model, effectively overcoming the limitations posed by the memory capacity of individual GPUs.

By the end of this assignment, you should be well-acquainted with these cutting-edge techniques and be capable of integrating them into your own machine learning projects. Let's embark on this exciting journey into the vanguard of machine learning fine-tuning methodologies!

### Dataset

The Stanford Alpaca dataset is part of a project that aims to build and share an instruction-following model called Alpaca. The dataset contains 52,000 examples used for fine-tuning the Alpaca model, with each example consisting of a unique instruction that the model should follow, an optional context or input for the task, and the corresponding output generated by the OpenAI's text-davinci-003 model. More information is available at the [Data release](https://github.com/tatsu-lab/stanford_alpaca/blob/main/README.md#data-release) and [Alpaca project page](https://crfm.stanford.edu/2023/03/13/alpaca.html).

### Model

The [Phi-2 Language Model from Hugging Face](https://www.microsoft.com/en-us/research/blog/phi-2-the-surprising-power-of-small-language-models/) represents a notable advancement in the field of AI, boasting 2.7 billion parameters and delivering top-tier performance among base language models with fewer than 13 billion parameters. It competes with or even exceeds the capabilities of models that are up to 25 times its size on various complex benchmarks.

Given our technical constraints and resource limitations, we find the [GPTQ quantization](https://arxiv.org/abs/2210.17323) method to be an optimal choice for training. This approach allows us to maintain high-quality output while reducing the model's bit size through precise fine-tuning.

We plan to apply GPTQ quantization to fine-tune the Phi-2-2.7B model, utilizing the Stanford Alpaca dataset. This approach mirrors the supervised fine-tuning phase used in developing models akin to ChatGPT, aiming to improve Phi-2-2.7B's proficiency in understanding instructions and executing tasks as directed by users.




### Initial setup

To prepare the environment for our project, please execute the commands below:

#### Clone and Install Libraries (Do not run if you haved)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ANLP04/Assignments/assignment02
!ls -la
!pip install -r requirements.txt

/content/drive/MyDrive/ANLP04/Assignments/assignment02
total 10849
-rw------- 1 root root 10948272 Feb  6 04:01  alpaca_data.json
drwx------ 2 root root     4096 Feb 26 09:40  checkpoints
drwx------ 2 root root     4096 Feb 16 07:49  dataset
drwx------ 2 root root     4096 Feb 26 09:40  epoch_1_checkpoint
drwx------ 2 root root     4096 Feb 26 09:41  epoch_2_checkpoint
drwx------ 2 root root     4096 Feb 26 09:41  epoch_3_checkpoint
-rw------- 1 root root     2472 Feb  6 04:01  inference.py
-rw------- 1 root root    13905 Feb 16 08:58  lora_layer.py
-rw------- 1 root root    32537 Feb  6 04:01  lora_model.py
-rw------- 1 root root    46617 Feb 26 09:41 '[NLP04-VietAI]-Assignment02.ipynb'
-rw------- 1 root root     1784 Feb 16 08:10  prepare_data.py
-rw------- 1 root root     1511 Feb  6 04:01  prompt.py
drwx------ 2 root root     4096 Feb 15 14:40  __pycache__
-rw------- 1 root root      237 Feb  6 04:01  requirements.txt
-rw------- 1 root root    16225 Feb  6 04:01  test_data.json
-rw

In [ ]:
# !git clone https://github.com/vietai-courses/Advanced-NLP04.git
# %cd Advanced-NLP04/assignment02
# !pip install -r requirements.txt

#### RUN ONLY AS NEEDED (Execute this code if you have recently restarted the kernel).

The following instructions are intended for situations where you encounter an Out of Memory error and need to restart the kernel. After restarting the kernel, execute the code in this cell to resume operations smoothly. Therefore, this should only be run if you've recently restarted the kernel.

In [ ]:
%cd Advanced-NLP04/assignment02

[Errno 2] No such file or directory: 'Advanced-NLP04/assignment02'
/content/drive/MyDrive/ANLP04/Assignments/assignment02


## Part 1: Low-Rank Adaptation of Large Language Models for Efficient Fine-tuning

![](https://miro.medium.com/v2/resize:fit:730/1*D_i25E9dTd_5HMa45zITSg.png)

Figure 1: LoRA method. We only train A and B.


### 1. Introduction

**LoRA, or Low-Rank Adaptation**, is a technique for adapting large language models to specific tasks or domains more efficiently. It's based on the observation that as models get larger, the conventional approach of full fine-tuning becomes less feasible due to the large number of parameters involved.

This process involves injecting the matrices into the dense layer's update, optimizing them for the specific adaptation task while the original pretrained model weights remain unchanged.

Here are some of the key points of the LoRA technique:

- **Freezing Pretrained Weights**: Instead of modifying all the parameters of a pretrained model during fine-tuning, LoRA freezes the pretrained weights. This means that the original model weights remain unchanged during the adaptation process.

- **Rank Decomposition Matrices**: LoRA freezes the pretrained model weights and *injects trainable rank decomposition matrices* into each layer of the Transformer architecture. These matrices are used to adjust the output of each layer in a way that's specific to the adaptation task.

- **Indirect Training of Dense Layers**: The rank decomposition matrices allow for the indirect training of each dense layer in the neural network. They are injected into the layer's update during the adaptation process and optimized to enhance the layer's performance on the specific task or domain.

- **Significant Reduction in Trainable Parameters**: By focusing on these rank decomposition matrices instead of the entire set of model weights, LoRA greatly reduces the number of trainable parameters for downstream tasks. For instance, in the case of GPT-3, LoRA can reduce the number of trainable parameters by a factor of 10,000.

- **Maintaining Model Performance**: Despite the significant reduction in the number of trainable parameters, the LoRA technique is designed to maintain or even improve the performance of the large language model on the specific task or domain.

In summary, LoRA is a method that tackles the challenge of adapting large language models to specific tasks or domains in a more efficient and feasible way, making the fine-tuning process more manageable and less resource-intensive.

### 2. Details

The LoRA technique introduces a mathematical concept known as low-rank approximation into the fine-tuning process of large language models. Here's a mathematical description of the process:

LoRA involves modifying the pre-trained weight matrix $\mathbf{W}_0 \in \mathbb{R}^{d \times k}$ of a neural network layer by introducing a low-rank parametrized update matrix $\Delta \mathbf{W} = \mathbf{B}\mathbf{A}$, where $\mathbf{B} \in \mathbb{R}^{d \times r}$, $\mathbf{A} \in \mathbb{R}^{r \times k}$, and $r \ll \min(d, k)$.

During the adaptation process, $\mathbf{W}_0$ is kept frozen, which means it does not receive any gradient updates. The trainable parameters are contained within $\mathbf{A}$ and $\mathbf{B}$, which form the low-rank update matrix $\Delta \mathbf{W}$.

It's important to note that both $\mathbf{W}_0$ and $\Delta \mathbf{W} = \mathbf{B}\mathbf{A}$ are multiplied with the same input, and their respective output vectors are summed. If $\mathbf{x}$ is the input and $\mathbf{h} = \mathbf{W}_0\mathbf{x}$ is the output of the original weight matrix, the modified output is:

$\mathbf{h} = \mathbf{W}_0\mathbf{x} + \Delta \mathbf{W} \mathbf{x} = \mathbf{W}_0\mathbf{x} + \mathbf{B}\mathbf{A}\mathbf{x} = (\mathbf{W}_0 + \mathbf{B}\mathbf{A})\mathbf{x}$

in which $\mathbf{W}_0 + \mathbf{B}\mathbf{A}$ is called **merge** operation. We will implement it in this assignment.

At the beginning of training, we initialize $\mathbf{A}$ with a random Gaussian distribution and $\mathbf{B}$ with zero, such that $\Delta \mathbf{W} = \mathbf{B}\mathbf{A}$ is zero, as shown in **Figure 1**. This ensures that the initial output of the model remains the same as in the pre-training phase, and the adaptation starts from the original model state.

The low-rank update $\Delta \mathbf{W} = \mathbf{B}\mathbf{A}$ then evolves during training, helping to specialize the model for a specific task while keeping the number of trainable parameters manageable. Additionally, $\Delta \mathbf{W}$ is scaled by $\frac{\alpha}{r}$ where $\alpha$ is a constant hyper-parameter. **(*)**

This process is applied for each Linear layer of self-attention layer in the BLOOM language model, leading to an adapted model that's specialized for a specific task or domain, with significantly fewer trainable parameters than the original model. For example, with GPT-3 175B, VRAM consumption during training is reduced from 1.2TB to 350GB. If $r = 4$ and only the query and value projection matrices are adapted, the checkpoint size is reduced by approximately 10,000 times (from 350GB to 35MB). This allows training with significantly fewer GPUs and helps to avoid communication overhead.

Another benefit is the ability to switch between tasks at a lower cost by only swapping the LoRA weights, as opposed to all parameters. This enables the creation of many customized models that can be swapped in and out on the fly on machines that store the pre-trained weights in VRAM.

**(*)** The reason for scaling the update $\Delta W x$ by $\frac{\alpha}{r}$ is primarily for easier optimization.

Consider the scenario where the rank $r$ changes during training. If you were to increase or decrease $r$, without this scaling factor, it would significantly affect the magnitude of the weight updates and thereby the learning dynamics of the model. In other words, changing $r$ would mean that you need to retune the learning rate or other hyperparameters, which is a laborious and time-consuming task.

By scaling the updates by $\frac{\alpha}{r}$, the authors make the learning process more robust to changes in $r$. $\alpha$ is a constant, so this scaling factor effectively normalizes the magnitude of the updates relative to the rank of the low-rank approximation.

This way, even when $r$ changes, the overall scale of the updates remains approximately constant, meaning you can use the same learning rate and other hyperparameters. This is advantageous because it makes the training process more efficient and less sensitive to the choice of $r$.

Keep in mind that this is a heuristic and it may not always provide the optimal solution for every problem or dataset, but it is a practical choice that often works well in practice.

### 3. Implementation

Let's break down the code, please take a look at the `lora_layer.py` file. The main components are:

- **LoraLayer** class: This is a base class that provides common functionality for both linear and embedding layers using the LoRA technique. It keeps track of LoRA parameters including the rank **r** and two sets of weights **lora_A** and **lora_B** (or **lora_embedding_A** and **lora_embedding_B** for the embedding layer). Two methods, **update_layer** and **update_layer_embedding**, are defined to update these parameters for linear and embedding layers, respectively.

- **Linear** and **Embedding** classes: These classes extend their corresponding PyTorch classes (**nn.Linear** and **nn.Embedding**) and the **LoraLayer** class. They initialize their superclasses as well as the LoRA parameters, and overwrite the **merge**, **unmerge**, and **forward** methods to implement the LoRA technique. The **merge** method combines the original weights of the layer with the LoRA weights, and **unmerge** undoes this operation. The **forward** method applies the layer operation either with or without the LoRA technique, depending on whether LoRA is enabled.

This assignment is heavily based on the internal codebase of 🤗 PEFT library. 🤗 PEFT, or **Parameter-Efficient Fine-Tuning (PEFT)**, is a library for efficiently adapting pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model’s parameters. Recent state-of-the-art PEFT techniques achieve performance comparable to that of full fine-tuning.

If you are new to PEFT, get started by reading the [Quicktour](https://huggingface.co/docs/peft/quicktour) guide and conceptual guides for [LoRA](https://huggingface.co/docs/peft/conceptual_guides/lora) methods.

#### Q1: Implement the Merge operation in LoRA (15 points)
In the provided `lora_layer.py` file, your task is to complete the `merge` method within the `Linear` class. As a useful reference, consider the already implemented `merge` method in the `Embedding` class. This should provide a clear guide on how to approach this task.

#### Q2: Implement the Forward Pass in LoRA (15 points)
In the provided `lora_layer.py` file, your task is to complete the `forward` method within the `Linear` class. As a useful reference, consider the already implemented `forward` method in the `Embedding` class. This should provide a clear guide on how to approach this task.

#### Q3: Construct the LoRA Model and Dataloaders for Training (20 points)
In the provided `train.py` file, your task is to complete the `load_pretrained_model` function and `prepare_dataloader`. The aforementioned PEFT's Quicktour guide and LoRA's conceptual guide can be useful references. Note, the specific details related to distributed training can be overlooked at this stage.

Once you've finished the implementation, it's time to train your LoRA model. Congratulations!

##### Train with sample data

In [ ]:
# train with sample dataset
!DEBUG=true python train.py

2024-02-16 09:02:37.693197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 09:02:37.693244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 09:02:37.694619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 09:02:38.802607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
trainable params: 430080 || all params: 262794240 || trainable%: 0.16365655502951662
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Completed to load config & tokenizer
L

##### Challenge 1: Will LoRA enhance inference speed? (5 points)

- No. LoRA will not enhance inference or introducing inference latency either.

##### Challenge 2: Will LoRA improve training speed? (5 points)

- Yes. Using LoRA will improve training speed compared to full fine-tuning as we do not need to calculate the gradient for the vast majority of the parameters.

## Part 2: Mixed precision training

The paper "Mixed Precision Training" is a game-changer in the world of deep learning. It introduces a method that combines different numerical precisions (like 32-bit and 16-bit) during model training. By using lower precision for certain parts of the training process, such as weight updates, we can speed up computations and reduce memory requirements without sacrificing accuracy. This technique leverages the increased computational power of modern GPUs and accelerators to achieve impressive results.

### Implementation

#### Q4: Implement Mixed Precision Training (15 points)
In the provided `train.py` file, your objective is to enable mixed precision training. To achieve this, complete the assignment of the `mixed_precision_dtype`, `self.ctx` and `self.gradscaler`. You may have to modify the `_run_batch` and `_run_epoch` using `self.gradscaler` in case you are using `mixed_precision_dtype` of `torch.float16`. If you paid close attention to the coding session during week 6, you should find this task straightforward.

Once you have carried out these steps, proceed to execute the following cell to train your LoRA model with mixed precision training. You should observe significant speed improvement in training.

##### Train with sample data

In [ ]:
# mixed precision training with sample dataset
!DEBUG=true ON_MP=true python train.py

2024-02-26 09:37:52.941663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 09:37:52.941720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 09:37:52.943085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 09:37:54.053910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
config.json: 100% 1.59k/1.59k [00:00<00:00, 8.02MB/s]
configuration_phi.py: 100% 2.03k/2.03k [00:00<00:00, 9.93MB/s]
A new version of the following files was downloaded from https://huggingface.co/TheBloke/phi-2-GPTQ:
- configuration_phi.py


## Part 3: Distributed Training with DistributedDataParallel

When it comes to training large language models, like those used for NLP tasks, the computational requirements can be ridiculously expensive. These models often have billions of parameters and require vast amounts of data to train effectively. This is where distributed training, and more specifically DistributedDataParallel (DDP), comes into play.

Training large language models on a single GPU can be extremely time-consuming and sometimes outright impossible due to memory limitations. DDP allows us to train these models across multiple GPUs, and even across several machines. This not only speeds up the process but also allows us to train much larger models than would be possible on a single GPU.

By dividing the model and the dataset across multiple GPUs, each with its own subset of data, we can train in parallel. This significantly reduces the time required to train these large models. Furthermore, the synchronization of model parameters after each forward and backward pass ensures consistency and accuracy across all model replicas.

In this sections, we will utilize DistributedDataParallel for training large language models. Let's dive in!

### Implementation

#### Q5: Setup environment for DDP (25 points)

In the provided `train.py` file, your mission is to enable distributed training utilizing the `DistributedDataParallel` (DDP) module from PyTorch. This task involves selecting the correct `distributed_strategy`, initializing the process group, establishing the local rank, and filling out the `_set_ddp_training` function. Furthermore, you are required to adapt the `load_pretrained_model` function and the `prepare_dataloader` method to be compatible with DDP training.

Once you have carried out these steps, complete the `torchrun` command below to execute the following cell to train your LoRA model on Kaggle GPU T4 x2.

##### Train with sample data

In [3]:
# distributed training with sample dataset

# Fill in blank "..."
!DEBUG=true ON_DDP=true ON_MP=true torchrun ... train.py

/usr/bin/python3: can't open file '/content/drive/MyDrive/ANLP04/Assignments/assignment02/...': [Errno 2] No such file or directory
[2024-02-26 14:36:08,185] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 2) local_rank: 0 (pid: 6918) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 806, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 797, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 134, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
  File "/usr/local/lib/python3.10/dist-packages

##### Train with full data

In [ ]:
# distributed training with full dataset

# Fill in blank "..."
!ON_DDP=true ON_MP=true torchrun ... train.py

### Inference

Once the training phase concludes, we can utilize the subsequent code to evaluate our model and generate some instructions. Let's give it a try!

In [ ]:
from inference import generate_inference

model_path = 'TheBloke/phi-2-GPTQ'

lora_weights_path = 'epoch_1_checkpoint' #you can edit it
instruction = 'Write about new house story'
user_inp = "My house is green" # TODO: fill input

outputs = generate_inference(instruction=instruction, user_inp=user_inp, model_path=model_path, lora_weights_path=lora_weights_path)
print(outputs)